In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [32]:
t1 = pd.read_excel('t1_norm.xlsx')
t1 = t1.select_dtypes(include=[np.number])
t1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 54 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   CO2_Emissions_1970            52 non-null     float64
 1   CO2_Emissions_1971            52 non-null     float64
 2   CO2_Emissions_1972            52 non-null     float64
 3   CO2_Emissions_1973            52 non-null     float64
 4   CO2_Emissions_1974            52 non-null     float64
 5   CO2_Emissions_1975            52 non-null     float64
 6   CO2_Emissions_1976            52 non-null     float64
 7   CO2_Emissions_1977            52 non-null     float64
 8   CO2_Emissions_1978            52 non-null     float64
 9   CO2_Emissions_1979            52 non-null     float64
 10  CO2_Emissions_1980            52 non-null     float64
 11  CO2_Emissions_1981            52 non-null     float64
 12  CO2_Emissions_1982            52 non-null     float64
 13  CO2_Emi

In [33]:
X = t1.drop(columns=['CO2_Emissions_2021'])
y = t1['CO2_Emissions_2021']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [36]:
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [37]:
cnn_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten()
])

/Users/admin/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [38]:
ann_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
])

/Users/admin/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [39]:
cnn_input = tf.keras.layers.Input(shape=(X_train_cnn.shape[1], 1))
ann_input = tf.keras.layers.Input(shape=(X_train.shape[1],))

In [40]:
cnn_output = cnn_model(cnn_input)
ann_output = ann_model(ann_input)

In [41]:
combined_output = tf.keras.layers.concatenate([cnn_output, ann_output])

In [42]:
combined_output = tf.keras.layers.Dense(32, activation='relu')(combined_output)
output = tf.keras.layers.Dense(1)(combined_output)

In [43]:
model = tf.keras.models.Model(inputs=[cnn_input, ann_input], outputs=output)

In [44]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [45]:
history = model.fit([X_train_cnn, X_train], y_train, epochs=100, batch_size=32, validation_split=0.0)

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0479  
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6324    
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5789 
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0060     
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0049 
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0912 
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0599     
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0758 
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0364 
Epoch 10/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0114 
Epoch 11/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.4172e-04
Epoch 12/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0018 
Epoch 13/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0087 
Epoch 14/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0159 
Epoch 15/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0132     


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.9044e-06 
Epoch 81/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.4371e-06 
Epoch 82/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.4084e-06 
Epoch 83/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5454e-06 
Epoch 84/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.0427e-06 
Epoch 85/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.5226e-06 
Epoch 86/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.1794e-06 
Epoch 87/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.6446e-06 
Epoch 88/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5435e-06 
Epoch 89/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3242e-06 
Epoch 90/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3402e-06 
Epoch 91/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4471e-06 
Epoch 92/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5815e-06 
Epoch 93/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.7699e-07 
Epoch 94/100
2/2 ━━━━━━━━━━━━━━━━

In [46]:
mse_e10 = model.evaluate([X_test_cnn, X_test], y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 5.9114e-05


In [48]:
predictions = model.predict([X_test_cnn, X_test])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


In [49]:
errors = y_test - predictions.flatten()
mae_e10 = np.mean(np.abs(errors))
mse_e10 = np.mean(errors**2)

In [50]:
from sklearn.metrics import r2_score
r2_e10 = r2_score(y_test, predictions)

In [51]:
print("Mean Squared Error:", mse_e10)
print("Mean Absolute Error:", mae_e10)
print("R-squared error:", r2_e10)

Mean Squared Error: 5.9114486886350014e-05
Mean Absolute Error: 0.0031785522411344786
R-squared error: 0.9542569725217322
